In [56]:
%pylab inline
import xarray as xr
import pandas as pd
import glob 
import os
from datetime import datetime
import scipy.interpolate as interp

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/pgarg/.conda/envs/tropics/lib/python3.9/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [33]:
colnames=['Pressure','x','y','z','a','b','c','d','e','f','g']
df = pd.read_excel('radiosondes.xlsx',names=colnames, header=None)
df

,Pressure,x,y,z,a,b,c,d,e,f,g
0,992.0,180.0,22.0,21.3,96.0,16.36,0.0,0.0,295.8,343.1,298.7
1,989.0,206.0,22.4,20.3,88.0,15.41,353.0,0.0,296.5,341.1,299.2
2,981.0,277.0,24.0,21.4,85.0,16.66,335.0,1.0,298.8,347.5,301.8
3,967.0,403.0,25.0,20.0,74.0,15.47,303.0,3.0,301.0,346.7,303.8
4,957.0,495.0,26.0,21.2,75.0,16.87,279.0,4.0,302.9,353.1,306.0
...,...,...,...,...,...,...,...,...,...,...,...
41638,6.6,34158.0,-42.5,NaN,NaN,NaN,85.0,41.0,968.1,NaN,968.1
41639,6.3,34442.0,-42.1,NaN,NaN,NaN,85.0,40.0,981.6,NaN,981.6
41640,6.1,34693.0,-41.7,NaN,NaN,NaN,84.0,39.0,993.6,NaN,993.6
41641,5.0,36050.0,-39.7,NaN,NaN,NaN,80.0,35.0,1060.8,NaN,1060.8


In [42]:
df.Pressure.max(skipna=True)

1015.0

In [37]:
grps = df.isna().all(axis=1).cumsum()


In [39]:
dfs = [df.dropna() for _, df in df.groupby(grps)]

In [51]:
pres_levels = np.flip(np.arange(200,1020,5.))

In [52]:
pres_levels

array([1015., 1010., 1005., 1000.,  995.,  990.,  985.,  980.,  975.,
        970.,  965.,  960.,  955.,  950.,  945.,  940.,  935.,  930.,
        925.,  920.,  915.,  910.,  905.,  900.,  895.,  890.,  885.,
        880.,  875.,  870.,  865.,  860.,  855.,  850.,  845.,  840.,
        835.,  830.,  825.,  820.,  815.,  810.,  805.,  800.,  795.,
        790.,  785.,  780.,  775.,  770.,  765.,  760.,  755.,  750.,
        745.,  740.,  735.,  730.,  725.,  720.,  715.,  710.,  705.,
        700.,  695.,  690.,  685.,  680.,  675.,  670.,  665.,  660.,
        655.,  650.,  645.,  640.,  635.,  630.,  625.,  620.,  615.,
        610.,  605.,  600.,  595.,  590.,  585.,  580.,  575.,  570.,
        565.,  560.,  555.,  550.,  545.,  540.,  535.,  530.,  525.,
        520.,  515.,  510.,  505.,  500.,  495.,  490.,  485.,  480.,
        475.,  470.,  465.,  460.,  455.,  450.,  445.,  440.,  435.,
        430.,  425.,  420.,  415.,  410.,  405.,  400.,  395.,  390.,
        385.,  380.,

In [60]:
def vertical_interpolate(vcoord_data, interp_var, interp_levels):
    """A function to interpolate sounding data from each station to
    every millibar. Assumes a log-linear relationship.

    Input
    -----
    vcoord_data : A 1D array of vertical level values (e.g., pressure from a radiosonde)
    interp_var : A 1D array of the variable to be interpolated to all pressure levels
    vcoord_interp_levels : A 1D array containing veritcal levels to interpolate to

    Return
    ------
    interp_data : A 1D array that contains the interpolated variable on the interp_levels
    """

    # Make veritcal coordinate data and grid level log variables
    lnp = np.log(vcoord_data)
    lnp_intervals = np.log(interp_levels)

    # Use numpy to interpolate from observed levels to grid levels
    interp_data = np.interp(lnp_intervals[::-1], lnp[::-1], interp_var[::-1])[::-1]

    # Mask for missing data (generally only near the surface)
    mask_low = interp_levels > vcoord_data[0]
    mask_high = interp_levels < vcoord_data[-1]
    interp_data[mask_low] = interp_var[0]
    interp_data[mask_high] = interp_var[-1]

    return interp_data

In [69]:
def create_new_df(df):
    df_interp = pd.DataFrame(index=range(len(pres_levels)),columns=test_df.keys())
    df_interp['Pressure'] = pres_levels
    df_sub = df.loc[df.Pressure>=200.]
    for key in df_sub.keys()[1:]:
        df_interp[key] = vertical_interpolate(df_sub.Pressure.values,df_sub[key].values,pres_levels)
        
    return df_interp
        

In [74]:
create_new_df(dfs[100])

,Pressure,x,y,z,a,b,c,d,e,f,g
0,1015.0,245.000000,23.200000,22.000000,93.000000,17.150000,195.000000,2.000000,297.300000,347.100000,300.300000
1,1010.0,245.000000,23.200000,22.000000,93.000000,17.150000,195.000000,2.000000,297.300000,347.100000,300.300000
2,1005.0,245.000000,23.200000,22.000000,93.000000,17.150000,195.000000,2.000000,297.300000,347.100000,300.300000
3,1000.0,245.000000,23.200000,22.000000,93.000000,17.150000,195.000000,2.000000,297.300000,347.100000,300.300000
4,995.0,245.000000,23.200000,22.000000,93.000000,17.150000,195.000000,2.000000,297.300000,347.100000,300.300000
...,...,...,...,...,...,...,...,...,...,...,...
159,220.0,11830.392473,-49.309603,-66.235699,12.513571,0.027808,324.609603,23.609603,345.026722,345.165762,345.026722
160,215.0,11981.146136,-50.245101,-68.766097,10.218505,0.014930,325.493001,23.753499,345.794401,345.894401,345.794401
161,210.0,12133.994040,-51.122097,-65.884539,15.856338,0.027459,326.745853,23.127074,346.796682,346.896682,346.796682
162,205.0,12290.120569,-51.914627,-74.255999,9.975687,0.016829,328.317059,22.560980,347.921941,348.021941,347.921941


In [78]:
list_interp = []
for i in range(len(dfs)):
    try:
        list_interp.append(create_new_df(dfs[i]))
        print(i)
    except:
        print("No Valid Values")
        pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [79]:
len(list_interp)

285

In [81]:
list_interp[10]

,Pressure,x,y,z,a,b,c,d,e,f,g
0,1015.0,180.000000,22.800000,21.300000,91.000000,16.330000,235.000000,3.000000,296.500000,343.700000,299.400000
1,1010.0,180.000000,22.800000,21.300000,91.000000,16.330000,235.000000,3.000000,296.500000,343.700000,299.400000
2,1005.0,180.000000,22.800000,21.300000,91.000000,16.330000,235.000000,3.000000,296.500000,343.700000,299.400000
3,1000.0,180.000000,22.800000,21.300000,91.000000,16.330000,235.000000,3.000000,296.500000,343.700000,299.400000
4,995.0,180.000000,22.800000,21.300000,91.000000,16.330000,235.000000,3.000000,296.500000,343.700000,299.400000
...,...,...,...,...,...,...,...,...,...,...,...
159,220.0,11774.872472,-48.480438,-60.854679,23.131226,0.047995,295.668288,37.465370,346.267072,346.460389,346.267072
160,215.0,11926.301414,-49.395109,-63.666447,18.727252,0.037832,299.055960,34.755232,347.113990,347.273430,347.113990
161,210.0,12081.293715,-50.331304,-66.544380,14.219646,0.027430,302.523349,31.981321,347.980837,348.105604,347.980837
162,205.0,12239.339276,-51.298536,-69.495608,9.720560,0.015735,305.000000,30.000000,348.855888,348.955888,348.855888


In [82]:
comb_interp = []
for i in range(len(list_interp)):
    comb_interp.append(list_interp[i].values)

In [83]:
arr_comb = np.stack(comb_interp,axis=2)

In [85]:
arr_comb.shape

(164, 11, 285)